# Pseudocode

Goal: To run validation routines on the classification pipeline. 

Procedure:
1. Authenticate
2. Retrieve Models
3. Prepare Unittest data - Images with known Labels
4. Classify and record output results
5. Build confusion matrix 
6. Create classification diagnostics

# 1. Preparation and Authenticate

In [1]:
import os
from watson_developer_cloud import VisualRecognitionV3
import arcgis
import requests
from bs4 import BeautifulSoup
import json
from IPython.display import Image
from IPython.core.display import HTML 

In [3]:
visual_recognition = VisualRecognitionV3(
    '2018-03-19',
    iam_api_key=os.environ['BLUEMIX_API_KEY'])

In [7]:
visual_recognition.list_classifiers(verbose=True)

{'classifiers': [{'classes': [{'class': 'low'},
    {'class': 'medium'},
    {'class': 'high'}],
   'classifier_id': 'Esri_Classification_CobbCounty_1256542849',
   'core_ml_enabled': True,
   'created': '2018-06-26T15:00:48.899Z',
   'name': 'Esri_Classification_CobbCounty_Cam128',
   'owner': '52d3c878-5f0a-44c6-b191-358d19be7bc5',
   'status': 'ready',
   'updated': '2018-06-26T15:00:48.899Z'},
  {'classes': [{'class': 'low'}, {'class': 'medium'}, {'class': 'high'}],
   'classifier_id': 'Esri_Classification_CobbCounty_371717976',
   'core_ml_enabled': True,
   'created': '2018-06-26T15:08:09.890Z',
   'name': 'Esri_Classification_CobbCounty_Cam130',
   'owner': '52d3c878-5f0a-44c6-b191-358d19be7bc5',
   'status': 'ready',
   'updated': '2018-06-26T15:08:09.890Z'},
  {'classes': [{'class': 'low'}, {'class': 'medium'}, {'class': 'high'}],
   'classifier_id': 'Esri_Classification_CobbCounty_361684145',
   'core_ml_enabled': True,
   'created': '2018-06-26T15:06:56.039Z',
   'name': 'Es

In [28]:
classifier_name = 'Camera_130_ViewA_Model'

In [9]:
for classifier in visual_recognition.list_classifiers()['classifiers']:
    classifier_id = None
    if classifier['name'] == classifier_name:
        classifier_id = classifier['classifier_id']

print(classifier_id)

Camera_130_ViewA_Model_766872466


In [10]:
images_url = r"https://www.cobbgis.org/openimage/bravescam"

In [11]:
cam = "130"

In [12]:
cam_url = "{0}/Cam{1}".format(images_url, cam)

In [13]:
cam_url

'https://www.cobbgis.org/openimage/bravescam/Cam130'

In [14]:
page = requests.get(cam_url)

In [15]:
soup = BeautifulSoup(page.content)

C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\ProgramData\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [16]:
links = []

In [17]:
for link in soup.findAll('a'):
    if link.get('href') != "/openimage/bravescam/":
        links.append(link.get('href'))

In [18]:
links

['/openimage/bravescam/Cam130/Cam_13018-06-21_14-20-19-86.jpg',
 '/openimage/bravescam/Cam130/Cam_13018-06-21_14-20-29-83.jpg',
 '/openimage/bravescam/Cam130/Cam_13018-06-21_14-20-39-86.jpg',
 '/openimage/bravescam/Cam130/Cam_13018-06-21_14-20-49-86.jpg',
 '/openimage/bravescam/Cam130/Cam_13018-06-21_14-20-59-83.jpg',
 '/openimage/bravescam/Cam130/Cam_13018-06-21_14-21-09-83.jpg',
 '/openimage/bravescam/Cam130/Cam_13018-06-21_14-21-19-83.jpg',
 '/openimage/bravescam/Cam130/Cam_13018-06-21_14-21-29-80.jpg',
 '/openimage/bravescam/Cam130/Cam_13018-06-21_14-21-39-83.jpg',
 '/openimage/bravescam/Cam130/Cam_13018-06-21_14-21-49-83.jpg',
 '/openimage/bravescam/Cam130/Cam_13018-06-21_14-21-59-86.jpg',
 '/openimage/bravescam/Cam130/Cam_13018-06-21_14-22-09-83.jpg',
 '/openimage/bravescam/Cam130/Cam_13018-06-21_14-22-19-83.jpg',
 '/openimage/bravescam/Cam130/Cam_13018-06-21_14-22-29-76.jpg',
 '/openimage/bravescam/Cam130/Cam_13018-06-21_14-22-39-83.jpg',
 '/openimage/bravescam/Cam130/Cam_13018-

In [19]:
test_image_url = "{0}{1}".format("https://www.cobbgis.org", links[-1])

In [20]:
test_image_url

'https://www.cobbgis.org/openimage/bravescam/Cam130/Cam_13018-06-25_22-24-19-80.jpg'

In [21]:
Image(url= test_image_url)

In [22]:
with open("pic_test.jpg", 'wb') as handle:
        response = requests.get(test_image_url, stream=True)

        if not response.ok:
            print(response)

        for block in response.iter_content(1024):
            if not block:
                break

            handle.write(block)

In [23]:
with open('pic_test.jpg', 'rb') as images_file:
    classes = visual_recognition.classify(
        images_file,
        threshold='0.15',
        classifier_ids=classifier_id)
    print(json.dumps(classes, indent=2))

{
  "images": [
    {
      "classifiers": [
        {
          "classifier_id": "Camera_130_ViewA_Model_766872466",
          "name": "Camera_130_ViewA_Model",
          "classes": [
            {
              "class": "Low",
              "score": 0.889
            }
          ]
        }
      ],
      "image": "pic_test.jpg"
    }
  ],
  "images_processed": 1,
  "custom_classes": 3
}


In [24]:
classes = visual_recognition.classify(
        test_image_url,
        threshold='0.15',
        classifier_ids=classifier_id)

print(json.dumps(classes, indent=2))

WatsonApiException: Error: No images were specified., Code: 400 , X-dp-watson-tran-id: gateway02-249287267 , X-global-transaction-id: ffea405d5b31a4000edbd263